# Vectorized Interchange representations

In [ ]:
import numpy
from openff.interchange import Interchange
from openff.toolkit import ForceField, Molecule
from rich.pretty import pprint

In [ ]:
sage = ForceField("openff_unconstrained-2.2.0.offxml")
molecule = Molecule.from_smiles(r"F\C=C/F")
interchange = Interchange.from_smirnoff(sage, [molecule])

pprint(interchange.collections.keys())

In [ ]:
molecule.visualize()

`Interchange`s from SMIRNOFF force fields contain collections for several different types of parameters. For simplicity, let's look at the bond collection.

In [ ]:
collection = interchange.collections["Bonds"]

`Collection`s store force field parameters and information about how they related to the topology they are applied to. In addition, they include some handy methods for transforming these to vectorized representations.

In [ ]:
# Parameters from the force field; one row per force field parameter

pprint(collection.get_force_field_parameters())

`Collection.get_force_field_parameters` returns an array with one row per unique force field parameter used and one colum per number in each parameter. For this molecule, that means three rows (C-F, C#C, and C-H chemistries) and two columns (`k` and `length`). This matrix scales with the number of unique force field parameters used so it will not generally scale with system size.

In [ ]:
# Parameters assigned to the system; one row per atom

pprint(collection.get_system_parameters())

`Collection.get_system_field_parameters` returns a similar array but with one row per bond in the topology, including duplicates. Since there are two C-H and two C-F bonds, those parameters each appear twice. This matrix scales with the size of the system (in this case, number of bonds) so it may be large for large systems.

In [ ]:
# Mapping from atoms to force field parameters; one row per value per atom
# bonds have lengths and force constants, so two rows per atom in this case

pprint(collection.get_param_matrix())

It may be useful to encode the relationships between force field parameters and where in the topology they're applied. This is handled by `collection.get_param_matrix()`, which returns a spare matrix. Each column corresponds to a force field parameter and each row corresponds to a bond that could be associated with each, each dimension being a flattened representation of the above matrices. A 1 indicates that a parameter is applied to that bond, a 0 indicates that it is not. For example, the 1 at `[0, 0]` indicates that the first bond gets assigned the first `k` value. The 1 at `[7, 1]` indicates that the fourth bond gets assigned the first `length`.

Conveniently, the dot product of this matrix with a flattened view of the force field parameters is equal to the view of the system parameters we saw above.

In [ ]:
dotted = numpy.dot(
    interchange["Bonds"].get_param_matrix(),
    interchange["Bonds"].get_force_field_parameters().flatten(),
).reshape((-1, 2))

assert numpy.allclose(dotted, collection.get_system_parameters())

pprint(dotted)